<a href="https://colab.research.google.com/github/dspierre/ConsoleApp2/blob/master/r%C3%A9vision_for_bpce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import os

In [2]:
d1 = pd.read_csv('/content/sample_data/Corona_NLP_test.csv')
d1.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [3]:
d1.isna().sum()

UserName           0
ScreenName         0
Location         834
TweetAt            0
OriginalTweet      0
Sentiment          0
dtype: int64

In [4]:
d1 = d1.dropna()

In [5]:
d1.isna().sum()

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [6]:
pd.unique(d1.Sentiment)

array(['Extremely Negative', 'Positive', 'Negative', 'Neutral',
       'Extremely Positive'], dtype=object)

In [7]:
pd.unique(d1.ScreenName)

array([44953, 44954, 44956, ..., 48747, 48748, 48750])

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
comment = d1["Sentiment"]
le.fit(comment)
list(le.classes_)
d1['Sentiment']=le.transform(d1['Sentiment'])
d1

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,0
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,4
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,2
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,3
5,6,44958,Los Angeles,03-03-2020,Do you remember the last time you paid $2.99 a...,3
...,...,...,...,...,...,...
3792,3793,48745,Washington D.C.,16-03-2020,"@RicePolitics @MDCounties Craig, will you call...",2
3793,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,4
3794,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,2
3795,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...,3


In [9]:
d1.drop(axis=1, columns='ScreenName')

,UserName,Location,TweetAt,OriginalTweet,Sentiment
0,1,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,0
1,2,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,4
3,4,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,2
4,5,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,3
5,6,Los Angeles,03-03-2020,Do you remember the last time you paid $2.99 a...,3
...,...,...,...,...,...
3792,3793,Washington D.C.,16-03-2020,"@RicePolitics @MDCounties Craig, will you call...",2
3793,3794,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,4
3794,3795,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,2
3795,3796,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...,3


In [10]:
d2=pd.read_csv('/content/sample_data/Corona_NLP_train.csv', sep=',', encoding='latin-1')
d2.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [11]:
d2.isna().sum()

UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [12]:
pd.unique(d2.Sentiment)

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

In [13]:
pd.unique(d2.Location)

array(['London', 'UK', 'Vagabonds', ..., 'Juba south sudan', 'OHIO',
       'i love you so much || he/him'], dtype=object)

In [14]:
d2 = d2.dropna()

In [15]:
d2.isna().sum()

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [16]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
comment = d2["Sentiment"]
le.fit(comment)
list(le.classes_)
d2['Sentiment']=le.transform(d2['Sentiment'])
d2

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,3
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,4
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,4
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,4
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,4
...,...,...,...,...,...,...
41147,44946,89898,"Brooklyn, NY",14-04-2020,YÂall really shitting that much more at home?...,2
41149,44948,89900,"Toronto, Ontario",14-04-2020,Still shocked by the number of #Toronto superm...,2
41150,44949,89901,OHIO,14-04-2020,I never that weÂd be in a situation &amp; wor...,4
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,3


In [17]:
df=(pd.concat([d1, d2], ignore_index = True))
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,0
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,4
2,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,2
3,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,3
4,6,44958,Los Angeles,03-03-2020,Do you remember the last time you paid $2.99 a...,3
...,...,...,...,...,...,...
35526,44946,89898,"Brooklyn, NY",14-04-2020,YÂall really shitting that much more at home?...,2
35527,44948,89900,"Toronto, Ontario",14-04-2020,Still shocked by the number of #Toronto superm...,2
35528,44949,89901,OHIO,14-04-2020,I never that weÂd be in a situation &amp; wor...,4
35529,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,3


In [18]:
df_drop= df.drop(['UserName','ScreenName','TweetAt'], axis=1)
df_drop

,Location,OriginalTweet,Sentiment
0,NYC,TRENDING: New Yorkers encounter empty supermar...,0
1,"Seattle, WA",When I couldn't find hand sanitizer at Fred Me...,4
2,Chicagoland,#Panic buying hits #NewYork City as anxious sh...,2
3,"Melbourne, Victoria",#toiletpaper #dunnypaper #coronavirus #coronav...,3
4,Los Angeles,Do you remember the last time you paid $2.99 a...,3
...,...,...,...
35526,"Brooklyn, NY",YÂall really shitting that much more at home?...,2
35527,"Toronto, Ontario",Still shocked by the number of #Toronto superm...,2
35528,OHIO,I never that weÂd be in a situation &amp; wor...,4
35529,"Wellington City, New Zealand",Airline pilots offering to stock supermarket s...,3


In [19]:
y=df_drop['Sentiment']
y

0        0
1        4
2        2
3        3
4        3
        ..
35526    2
35527    2
35528    4
35529    3
35530    2
Name: Sentiment, Length: 35531, dtype: int64

In [20]:
X = df_drop.drop(columns=['Sentiment'])
X

,Location,OriginalTweet
0,NYC,TRENDING: New Yorkers encounter empty supermar...
1,"Seattle, WA",When I couldn't find hand sanitizer at Fred Me...
2,Chicagoland,#Panic buying hits #NewYork City as anxious sh...
3,"Melbourne, Victoria",#toiletpaper #dunnypaper #coronavirus #coronav...
4,Los Angeles,Do you remember the last time you paid $2.99 a...
...,...,...
35526,"Brooklyn, NY",YÂall really shitting that much more at home?...
35527,"Toronto, Ontario",Still shocked by the number of #Toronto superm...
35528,OHIO,I never that weÂd be in a situation &amp; wor...
35529,"Wellington City, New Zealand",Airline pilots offering to stock supermarket s...


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train

,Location,OriginalTweet
18836,"Kampala, Uganda",Let me stock food in time. Addressing the Nati...
16097,South Africa,@WOOLWORTHS_SA @the_dti @HealthZA I see Woolw...
26439,connecticut,going to the grocery store is now called going...
19640,Los Angeles,Delivery drivers are worried about their expos...
7024,North Devon,What will result from consumers being home-bou...
...,...,...
16850,"England, United Kingdom",The @cfo report that the price of light crude ...
6265,"Birmingham, England","What the hell is going in this country, shops ..."
11284,"London, England",I hope https://t.co/PDDzRQ4iio can help some p...
860,London,This is what selfish greedy people's homes loo...


In [23]:
X_test

,Location,OriginalTweet
32648,Mumbai,Indian govt will benefit because of low #oilpr...
9779,kisumu,"Supply chain woes due to these covid 19 ,healt..."
21614,"San Antonio, Texas","""How the private market responds to this crisi..."
19718,"Arizona, USA",COVID-19 consumer behaviour and B2B reinvest i...
18502,Maine,U S Retail Consumer Trends Shift Dramatically ...
...,...,...
2617,disrespected piscataway lands,When you re stocking up on supplies at the gro...
28367,"Corsicana, Texas","Due to the COVID-19 situation, food banks acro..."
11365,"New Jersey, USA",#ConsumerAlert: #Coronavirus #phishing scams r...
22085,Australia,Online Delivery Info - we understand that ther...


In [24]:
y_train

18836    4
16097    4
26439    3
19640    2
7024     3
        ..
16850    0
6265     2
11284    4
860      2
15795    4
Name: Sentiment, Length: 28424, dtype: int64

In [25]:
y_test

32648    4
9779     0
21614    0
19718    3
18502    0
        ..
2617     3
28367    4
11365    2
22085    1
3914     2
Name: Sentiment, Length: 7107, dtype: int64